# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=4):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    
    #Generates a larger line using hough_lines 
    extrapolated_lines = extrapolate_hough_lines(lines)
    
    
    for x1,y1,x2,y2  in extrapolated_lines:
        cv2.line(img, (x1, y1), (x2, y2), color, thickness)
                
            

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)


def calculate_slope_b(line):
    x1,y1,x2,y2 = line
    # Equation for a line y = m*x + b
    #calculates slope for each line
    if (x2-x1) != 0:
        slope = (y2-y1)/(x2-x1)
    else : 
        slope = float('inf')
    
    #calculates b for each line
    b = y1 - slope*x1 
    return slope, b

def is_equal(segment1, segment2, slope_tolerance = 0.1, b_tolerance = 10):
    slope_segment1, b_segment1= calculate_slope_b(segment1)
    slope_segment2, b_segment2= calculate_slope_b(segment2)
        
    diff_m = abs(slope_segment1 - slope_segment2)
    #print abs(diff_m)
    #calculates the diference between the slopes of the two lines recived.
    if diff_m <= slope_tolerance:
        #calculates the diference between the two b's recived.
        diff_b = abs(b_segment1 - b_segment2)
        #print diff_b
        if diff_b <= b_tolerance:
            #if slope and b are equal then find the new points for the larger line that joints 2 small line
            return True

    return False


def calculate_new_line(segment_1, segment_2):    
    
    x1,y1,x2,y2 = segment_1
    x3,y3,x4,y4 = segment_2
   
    point_1 = ((x1,y1),(x2,y2))
    point_2 = ((x3,y3),(x4,y4))
    four_points = point_1 + point_2
    
    xmin, ymin = x1,y1
    xmax, ymax = x1,y1
            
    for x,y in four_points:
        if x < xmin:
            xmin = x
            ymin = y
            
        if x > xmax:
            xmax = x
            ymax = y
            
    new_line = [xmin,ymin,xmax,ymax]
            
    return new_line


def merge(segment, lines_list):
    for index in range (len(lines_list)):
        line = lines_list[index]
        equal = is_equal(segment, line)
    
        if equal:
            new_line = calculate_new_line(segment, line)
            lines_list[index] = new_line
            
            return True 
        
    lines_list.append(segment)
    return False
    

    
    
def extrapolate_hough_lines(lines):
    
    final_lines_list = []

    for line_list in lines:
        for x0,y0,x1,y1 in line_list:             
            merge(list([x0,y0,x1,y1]), final_lines_list )
            
    return final_lines_list
                


## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

In [ ]:
#Pipeline

def findingLaneLines(image):
    
    #printing out some stats
    #print('This image is:', type(image), 'with dimesions:', image.shape)
    ysize = image.shape[0]
    xsize = image.shape[1]
    imshape = image.shape
    #print (xsize,ysize)
    
    #Configuration for gaussian_blur
    kernel_size = 5
    
    #Configuration for Canny
    low_threshold = 40
    high_threshold = 150
       
    
    #Define region of interest
    left_bottom = (120, ysize-1)
    #left_upper = (455, 320)
    left_upper = (400, 340)
    #right_upper = (515, 320)
    right_upper = (575, 340)
    right_bottom = (910, ysize-1)
    
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 45     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 120 #minimum number of pixels making up a line
    max_line_gap = 100    # maximum gap in pixels between connectable line segments
    
    # List with coordinates for ROI definition
    vertices =np.array([[left_bottom,left_upper,right_upper,right_bottom]],dtype=np.int32)
    
    #Image in gray color
    gray = grayscale(image)
    
    # Apply Gaussian smoothing
    blur_gray = gaussian_blur(gray, kernel_size)
    
    #Detect edges using Canny method
    edges = canny(blur_gray, low_threshold, high_threshold)
    
    #ROI overlay on image
    roi = region_of_interest(edges, vertices)
    #roi = region_of_interest(gray, vertices)
    #plt.imshow(roi)
        
    #Detect lines in ROI
    lines = hough_lines(roi, rho, theta, threshold, min_line_length, max_line_gap)
    
  
    #plt.imshow(lines, cmap='gray')
    #plt.imshow(roi, cmap='gray')
    result_image = weighted_img(lines, image, α=0.8, β=1., λ=0.)

    
    return result_image


images = os.listdir("test_images/")
image_name = images[1]
path = 'test_images/'+ image_name
image_read = mpimg.imread(path)
final_image = findingLaneLines(image_read)
#plt.imshow(roi)

run your solution on all test_images and make copies into the test_images directory).

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    
    result = findingLaneLines(image)
    return result



Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


This is awesome! It was very exciting to do this project. In order to make my algorithm more robust, I imagine I should consider a calibration algorithm to find more accurate thresholds for Canny Edge Detection. 

I think my algorithm is likely to fail when the contrast between lanes and road is too low, since I won't be able to detect the edges of the lanes in gray scale. Another sceneario where my algorith will fail is when it counter sharp curves that fall out of the region of interest. 

Looking ahead, for this algorithm to be useful I need to redeploy it on a real time system, that can be mounted in the car.

## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))